<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-1">Load Data</a></span></li><li><span><a href="#Beautiful-Soup" data-toc-modified-id="Beautiful-Soup-2">Beautiful Soup</a></span></li></ul></div>

# Web Scraper
- Beautiful Soup: https://realpython.com/beautiful-soup-web-scraper-python/

In [1]:
import pandas as pd
import time, datetime
import multiprocessing as mp
from datetime import datetime as dt
import math

## Load Data

In [2]:
bad_cols = [0,2,6,15,16,17,18,19,20,21,22,24,25,26] # unnecessary columns from example_data notebook
cols = list(range(0,26)) # list of all columns
good_cols = [i for i in cols if i not in bad_cols] # create list of useful columns
good_cols = [str(x) for x in good_cols] # convert to string
good_cols

['1', '3', '4', '5', '7', '8', '9', '10', '11', '12', '13', '14', '23']

In [3]:
start = time.time()
# read data in chunks of 100k rows at a time
chunk = pd.read_csv('../../00-data/gdelt_data/2021/gdelt_sum_dec2021.csv.zip', 
                    chunksize=100000, 
                    dtype={'3': 'str'})#, # mixed data types
                    #index_col=[0]) # specify index column
# iter_csv = pd.read_csv('file.csv', iterator=True, chunksize=1000)
# df = pd.concat([chunk[chunk['field'] > constant] for chunk in iter_csv])
df = pd.concat(chunk) # concat chunks together
end = time.time()
print("Read csv with chunks: ",(end-start),"sec")

Read csv with chunks:  7.019579887390137 sec


In [4]:
df

,1,3,4,5,7,8,9,10,11,12,13,14,23
0,20211201000000,mcdowellnews.com,https://mcdowellnews.com/news/national/govt-an...,NaN,NaN,NaN,"2#Alaska, United States#US#USAK#61.385#-152.26...","2#Alaskans, United States#US#USAK##61.385#-152...",mike dunleavy;christine oconnor,"Mike Dunleavy,586;Christine Oconnor,256",alaska telecom association,"Alaska Telecom Association,205","Alaska Telecom Association,225;Mike Dunleavy,616"
1,20211201000000,observer.ug,https://www.observer.ug/viewpoint/72021-why-sh...,NaN,GENERAL_GOVERNMENT;EPU_POLICY;EPU_POLICY_GOVER...,"WB_178_PEST_MANAGEMENT,4266;WB_174_CROP_PRODUC...",1#Uganda#UG#UG#2#33#UG,1#Uganda#UG#UG##2#33#UG#204;1#Uganda#UG#UG##2#...,isabella akiteng,"Isabella Akiteng,949",friedrich ebert stiftung youth leaders program...,Friedrich Ebert Stiftung Youth Leaders Program...,"Friedrich Ebert Stiftung,902;Youth Leaders,917..."
2,20211201000000,menafn.com,https://menafn.com/1103282515/9-injured-in-ref...,WOUND#9##1#Algeria#AG#AG#28#3#AG;CRISISLEX_CRI...,WOUND;CRISISLEX_CRISISLEXREC;CRISISLEX_C03_WEL...,"LEADER,524;ENV_OIL,96;ENV_MINING,539;DISASTER_...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#36...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#38...",mohamed arkab,"Mohamed Arkab,553",sonatrach;menafn- trend news agency;sonatrach ...,"Sonatrach,295;Sonatrach,344;Sonatrach,714;Mena...","News Agency,23;Sonatrach Group,299;Algerian Mi..."
3,20211201000000,hot1063.com,https://www.hot1063.com/news/red-notice-is-now...,NaN,CRIME_COMMON_ROBBERY;TAX_FNCACT;TAX_FNCACT_THI...,"CRIME_COMMON_ROBBERY,727;TAX_FNCACT_THIEF,727;...",NaN,NaN,gal gadot;ryan reynolds;dwayne johnson,"Gal Gadot,417;Ryan Reynolds,405;Dwayne Johnson,14",netflix;warnermedia company;cable news network...,"Netflix,147;Netflix,251;Netflix,607;Warnermedi...","Dwayne Johnson,15;Sandra Bullock,340;Ryan Reyn..."
4,20211201000000,altenergymag.com,https://www.altenergymag.com/company_directory...,NaN,ENV_SOLAR;,"ENV_SOLAR,75;ENV_SOLAR,313;ENV_SOLAR,353;ENV_S...",1#United States#US#US#39.828175#-98.5795#US,1#America#US#US##39.828175#-98.5795#US#626;1#U...,NaN,NaN,company sector;free company listing;company ca...,"Company Sector,785;Free Company Listing,901;Co...","Project Development Support,94;Pima Center Pkw..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150669,20211201234500,freightwaves.com,https://www.freightwaves.com/news/thanksgiving...,NaN,NEGOTIATIONS;GEN_HOLIDAY;USPEC_POLICY1;EPU_POL...,"GEN_HOLIDAY,523;GEN_HOLIDAY,1080;GEN_HOLIDAY,1...",1#United States#US#US#39.828175#-98.5795#US;3#...,"2#Texas, United States#US#USTX##31.106#-97.647...",los angeles;kevin hill;tony mulvey,"Los Angeles,9113;Los Angeles,12066;Los Angeles...",walmart;freightwaves freight intel group;rate ...,"Walmart,3452;Freightwaves Freight Intel Group,...","Chain Pricing Power Index,46;Chain Pricing Pow..."
150670,20211201234500,newyorktelegraph.com,https://www.newyorktelegraph.com/news/27184719...,NaN,TAX_FNCACT;TAX_FNCACT_WORKERS;USPEC_POLICY1;EP...,"GENERAL_GOVERNMENT,212;EPU_POLICY_GOVERNMENT,2...","1#Germany#GM#GM#51.5#10.5#GM;4#Paris, France (...","4#Petersburg, Sankt-Peterburg, Russia#RS#RS66#...",peng shuai;queen elizabeth,"Peng Shuai,599;Queen Elizabeth,752",women tennis association;international section,"Women Tennis Association,522;International Sec...","Tennis Association,529;Peng Shuai,612;Queen El..."
150671,20211201234500,dailyrecord.co.uk,https://www.dailyrecord.co.uk/lifestyle/warnin...,NaN,TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_DOG;TAX_WORL...,"AGRICULTURE,4612;WB_1467_EDUCATION_FOR_ALL,307...",1#Sweden#SW#SW#62#15#SW;1#United Kingdom#UK#UK...,1#United Kingdom#UK#UK##54#-4#UK#202;1#United ...,carolyn menteith,"Carolyn Menteith,872",NaN,NaN,"United Kingdom,215;Agria Pet,282;Team Dogs,323..."
150672,20211201234500,scmp.com,https://www.scmp.com/news/china/diplomacy/arti...,NaN,GENERAL_HEALTH;MEDICAL;CRISISLEX_C07_SAFETY;UN...,"UNGP_FORESTS_RIVERS_OCEANS,115;GENERAL_HEALTH,...",1#China#CH#CH#35#105#CH,1#China#CH#CH##35#105#CH#23,NaN,NaN,reuters,"R

In [5]:
bad_rows = df[~df['1'].str.get(0).isin(['2'])].index # remove rows with dates that don't start with 2
bad_rows = bad_rows.append(df[~df['1'].str.get(1).isin(['0'])].index) # remove rows with dates second number isn't 0
df = df.drop(index = bad_rows)
df['1'] = [int(float(i)) for i in df['1']]
df

,1,3,4,5,7,8,9,10,11,12,13,14,23
0,20211201000000,mcdowellnews.com,https://mcdowellnews.com/news/national/govt-an...,NaN,NaN,NaN,"2#Alaska, United States#US#USAK#61.385#-152.26...","2#Alaskans, United States#US#USAK##61.385#-152...",mike dunleavy;christine oconnor,"Mike Dunleavy,586;Christine Oconnor,256",alaska telecom association,"Alaska Telecom Association,205","Alaska Telecom Association,225;Mike Dunleavy,616"
1,20211201000000,observer.ug,https://www.observer.ug/viewpoint/72021-why-sh...,NaN,GENERAL_GOVERNMENT;EPU_POLICY;EPU_POLICY_GOVER...,"WB_178_PEST_MANAGEMENT,4266;WB_174_CROP_PRODUC...",1#Uganda#UG#UG#2#33#UG,1#Uganda#UG#UG##2#33#UG#204;1#Uganda#UG#UG##2#...,isabella akiteng,"Isabella Akiteng,949",friedrich ebert stiftung youth leaders program...,Friedrich Ebert Stiftung Youth Leaders Program...,"Friedrich Ebert Stiftung,902;Youth Leaders,917..."
2,20211201000000,menafn.com,https://menafn.com/1103282515/9-injured-in-ref...,WOUND#9##1#Algeria#AG#AG#28#3#AG;CRISISLEX_CRI...,WOUND;CRISISLEX_CRISISLEXREC;CRISISLEX_C03_WEL...,"LEADER,524;ENV_OIL,96;ENV_MINING,539;DISASTER_...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#36...","4#Skikda, Wilaya De Skikda, Algeria#AG#AG31#38...",mohamed arkab,"Mohamed Arkab,553",sonatrach;menafn- trend news agency;sonatrach ...,"Sonatrach,295;Sonatrach,344;Sonatrach,714;Mena...","News Agency,23;Sonatrach Group,299;Algerian Mi..."
3,20211201000000,hot1063.com,https://www.hot1063.com/news/red-notice-is-now...,NaN,CRIME_COMMON_ROBBERY;TAX_FNCACT;TAX_FNCACT_THI...,"CRIME_COMMON_ROBBERY,727;TAX_FNCACT_THIEF,727;...",NaN,NaN,gal gadot;ryan reynolds;dwayne johnson,"Gal Gadot,417;Ryan Reynolds,405;Dwayne Johnson,14",netflix;warnermedia company;cable news network...,"Netflix,147;Netflix,251;Netflix,607;Warnermedi...","Dwayne Johnson,15;Sandra Bullock,340;Ryan Reyn..."
4,20211201000000,altenergymag.com,https://www.altenergymag.com/company_directory...,NaN,ENV_SOLAR;,"ENV_SOLAR,75;ENV_SOLAR,313;ENV_SOLAR,353;ENV_S...",1#United States#US#US#39.828175#-98.5795#US,1#America#US#US##39.828175#-98.5795#US#626;1#U...,NaN,NaN,company sector;free company listing;company ca...,"Company Sector,785;Free Company Listing,901;Co...","Project Development Support,94;Pima Center Pkw..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150669,20211201234500,freightwaves.com,https://www.freightwaves.com/news/thanksgiving...,NaN,NEGOTIATIONS;GEN_HOLIDAY;USPEC_POLICY1;EPU_POL...,"GEN_HOLIDAY,523;GEN_HOLIDAY,1080;GEN_HOLIDAY,1...",1#United States#US#US#39.828175#-98.5795#US;3#...,"2#Texas, United States#US#USTX##31.106#-97.647...",los angeles;kevin hill;tony mulvey,"Los Angeles,9113;Los Angeles,12066;Los Angeles...",walmart;freightwaves freight intel group;rate ...,"Walmart,3452;Freightwaves Freight Intel Group,...","Chain Pricing Power Index,46;Chain Pricing Pow..."
150670,20211201234500,newyorktelegraph.com,https://www.newyorktelegraph.com/news/27184719...,NaN,TAX_FNCACT;TAX_FNCACT_WORKERS;USPEC_POLICY1;EP...,"GENERAL_GOVERNMENT,212;EPU_POLICY_GOVERNMENT,2...","1#Germany#GM#GM#51.5#10.5#GM;4#Paris, France (...","4#Petersburg, Sankt-Peterburg, Russia#RS#RS66#...",peng shuai;queen elizabeth,"Peng Shuai,599;Queen Elizabeth,752",women tennis association;international section,"Women Tennis Association,522;International Sec...","Tennis Association,529;Peng Shuai,612;Queen El..."
150671,20211201234500,dailyrecord.co.uk,https://www.dailyrecord.co.uk/lifestyle/warnin...,NaN,TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_DOG;TAX_WORL...,"AGRICULTURE,4612;WB_1467_EDUCATION_FOR_ALL,307...",1#Sweden#SW#SW#62#15#SW;1#United Kingdom#UK#UK...,1#United Kingdom#UK#UK##54#-4#UK#202;1#United ...,carolyn menteith,"Carolyn Menteith,872",NaN,NaN,"United Kingdom,215;Agria Pet,282;Team Dogs,323..."
150672,20211201234500,scmp.com,https://www.scmp.com/news/china/diplomacy/arti...,NaN,GENERAL_HEALTH;MEDICAL;CRISISLEX_C07_SAFETY;UN...,"UNGP_FORESTS_RIVERS_OCEANS,115;GENERAL_HEALTH,...",1#China#CH#CH#35#105#CH,1#China#CH#CH##35#105#CH#23,NaN,NaN,reuters,"R

In [6]:
filter_df = df[df['3']=='prnewswire.com']
filter_df

,1,3,4,5,7,8,9,10,11,12,13,14,23
178,20211201000000,prnewswire.com,https://www.prnewswire.com:443/news-releases/c...,NaN,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"ECON_WORLDCURRENCIES_DOLLAR,5483;WB_678_DIGITA...","2#New York, United States#US#USNY#42.1497#-74....","4#Dublin, Dublin, Ireland#EI#EI07#18288#53.333...",NaN,NaN,cumulative distribution rate for the fund;fund...,"Cumulative Distribution Rate For The Fund,5428...","Steers Quality Income Realty Fund,145;Capital ..."
483,20211201000000,prnewswire.com,https://www.prnewswire.com:443/news-releases/c...,NaN,WB_336_NON_BANK_FINANCIAL_INSTITUTIONS;WB_342_...,"ECON_WORLDCURRENCIES_DOLLAR,5348;WB_678_DIGITA...","2#New York, United States#US#USNY#42.1497#-74....","4#Dublin, Dublin, Ireland#EI#EI07#18288#53.333...",NaN,NaN,cumulative distribution rate for the fund;fund...,"Cumulative Distribution Rate For The Fund,5293...","Steers Infrastructure Fund,138;Capital Source,..."
1059,20211201000000,prnewswire.com,https://www.prnewswire.com:443/news-releases/c...,NaN,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"ECON_WORLDCURRENCIES_DOLLAR,5213;WB_678_DIGITA...","2#New York, United States#US#USNY#42.1497#-74....","4#Dublin, Dublin, Ireland#EI#EI07#18288#53.333...",NaN,NaN,cumulative distribution rate for the fund;inco...,"Cumulative Distribution Rate For The Fund,5158...","Steers Select Preferred,143;Income Fund,159;Ca..."
1328,20211201000000,prnewswire.com,https://www.prnewswire.com:443/news-releases/c...,NaN,USPEC_POLICY1;EPU_POLICY;EPU_POLICY_POLICY;TAX...,"ECON_WORLDCURRENCIES_DOLLAR,5494;WB_678_DIGITA...","2#New York, United States#US#USNY#42.1497#-74....","4#Dublin, Dublin, Ireland#EI#EI07#18288#53.333...",NaN,NaN,cumulative distribution rate for the fund;real...,"Cumulative Distribution Rate For The Fund,5439...","Steers Total Return Realty Fund,151;Capital So..."
1506,20211201000000,prnewswire.com,https://www.prnewswire.com:443/news-releases/d...,NaN,CRISISLEX_C07_SAFETY;PROTEST;WB_2670_JOBS;WB_2...,"PROTEST,312;PROTEST,626;EPU_CATS_MIGRATION_FEA...",NaN,NaN,ilan rabinovitch,"Ilan Rabinovitch,880",datadog inc;exchange commission on;amazon web ...,"Exchange Commission On,3658;Amazon Web Service...","Amazon Web Services,259;Service Ready,281;Read..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
149378,20211201234500,prnewswire.com,https://www.prnewswire.com:443/news-releases/l...,NaN,AGRICULTURE;TAX_FNCACT;TAX_FNCACT_EMPLOYEES;MI...,"AGRICULTURE,203;AGRICULTURE,1518;MILITARY,469;...","2#Ohio, United States#US#USOH#40.3736#-82.7755...","2#Michigan, United States#US#USMI##43.3504#-84...",lou burr,"Lou Burr,1013",u s army,NaN,"Lou Burr,1118"
149495,20211201234500,prnewswire.com,https://www.prnewswire.com:443/news-releases/c...,NaN,EPU_CATS_REGULATION;TRIAL;TAX_FNCACT;TAX_FNCAC...,"WB_837_OPEN_GOVERNMENT_AND_TRANSPARENCY,1837;W...","2#New York, United States#US#USNY#42.1497#-74....","2#New York, United States#US#USNY##42.1497#-74...",prnewswire claimsfiler,"Prnewswire Claimsfiler,37",lawyers at kahn swick foti;peloton interactive...,"Lawyers At Kahn Swick Foti,577;United States D...","Peloton Interactive,301;United States District..."
149932,20211201234500,prnewswire.com,https://www.prnewswire.com:443/news-releases/c...,NaN,EPU_CATS_REGULATION;TRIAL;TAX_FNCACT;TAX_FNCAC...,"WB_837_OPEN_GOVERNMENT_AND_TRANSPARENCY,1854;W...","3#Stoneco, New York, United States#US#USNY#41....","2#New York, United States#US#USNY##42.1497#-74...",prnewswire claimsfiler,"Prnewswire Claimsfiler,37",stoneco ltd;lawyers at kahn swick foti;united ...,"Lawyers At Kahn Swick Foti,591;United States D...","United States District Court,518;Southern Dist..."
150194,20211201234500,prnewswire.com,https://www.prnewswire.com:443/news-releases/a...,NaN,EDUCATION;SOC_POINTSOFINTEREST;SOC_POINTSOFINT...,"EPU_ECONOMY_HISTORIC,1180;TAX_FNCACT_DIRECTOR,...",NaN,NaN,dorna werdelin;ian toppin;delta techops;joseph...,"Dorna Werdelin,2283;Ian Toppin,1155;Delta Tech...",atlanta technical college;aviation

In [7]:
filter_urls = filter_df['4']
filter_urls

178       https://www.prnewswire.com:443/news-releases/c...
483       https://www.prnewswire.com:443/news-releases/c...
1059      https://www.prnewswire.com:443/news-releases/c...
1328      https://www.prnewswire.com:443/news-releases/c...
1506      https://www.prnewswire.com:443/news-releases/d...
                                ...                        
149378    https://www.prnewswire.com:443/news-releases/l...
149495    https://www.prnewswire.com:443/news-releases/c...
149932    https://www.prnewswire.com:443/news-releases/c...
150194    https://www.prnewswire.com:443/news-releases/a...
150494    https://www.prnewswire.com/news-releases/light...
Name: 4, Length: 1737, dtype: object

In [8]:
filter_urls.iloc[1]

'https://www.prnewswire.com:443/news-releases/cohen--steers-infrastructure-fund-inc-utf-notification-of-sources-of-distribution-under-section-19a-301434537.html'

## Beautiful Soup

In [15]:
from bs4 import BeautifulSoup
import requests

URL = filter_urls.iloc[1]
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

In [16]:
soup

<!DOCTYPE html>

<html lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/><link href="/content/dam/prnewswire/icons/2019-Q4-PRN-Icon-32-32.png" rel="icon" type="image/png"/>
<meta content="//assets.adobedtm.com/launch-ENb957f6e7383d4549b191e1190ed5476e.min" name="adobeLaunchScript"/> <meta content="us-prod" name="optanonhelper"/>
<script src="/etc.clientlibs/prncom/clientlibs/clientlib-library.min.js" type="text/javascript"></script>
<script src="/etc.clientlibs/prncom/clientlibs/component-clientlibs.min.js" type="text/javascript"></script>
<script src="https://widgets.financialcontent.com/cision/stockquote.js" type="text/javascript"></script>
<link href="/etc.clientlibs/prncom/clientlibs/newspage-clientlibs.min.css" rel="stylesheet" type="text/css"/>
<meta content='/PRNewswire/ -- This press release provides shareholders of Cohen &amp; Steers Infrastructure Fund,

In [18]:
master = soup.find(id="main")
print(master.prettify())

<main id="main" role="main">
 <script src="/etc.clientlibs/prncom/clientlibs/share_clientlibs.min.js" type="text/javascript">
 </script>
 <article class="news-release carousel-template">
  <header class="container release-header">
   <div class="custom-container">
    <div class="row">
     <div class="col-sm-12">
      <div class="row detail-headline">
       <div class="col-sm-12 col-xs-12">
        <h1>
         Cohen &amp; Steers Infrastructure Fund, Inc. (UTF) Notification of Sources of Distribution Under Section 19(a)
        </h1>
       </div>
      </div>
     </div>
    </div>
    <div class="row">
     <div class="col-sm-12">
      <hr/>
     </div>
    </div>
    <div class="row">
     <div class="col-lg-8 col-sm-5 swaping-class-left">
      <p class="meta">
       News provided by
      </p>
      <a href="/news/cohen-%26-steers%2C-inc./">
       <strong>
        Cohen &amp; Steers, Inc.
        <span aria-hidden="true" class="btb bt-long-arrow-right">
        </span>
    

In [19]:
final = master.find("div", class_="col-sm-10 col-sm-offset-1")
print(final.text)


NEW YORK, Nov. 30, 2021 /PRNewswire/ -- This press release provides shareholders of Cohen & Steers Infrastructure Fund, Inc. (NYSE: UTF) (the "Fund") with information regarding the sources of the distribution to be paid on November 30, 2021 and cumulative distributions paid fiscal year-to-date.
In March 2015, the Fund implemented a managed distribution policy in accordance with exemptive relief issued by the Securities and Exchange Commission. The managed distribution policy seeks to deliver the Fund's long-term total return potential through regular monthly distributions declared at a fixed rate per common share. This policy gives the Fund greater flexibility to realize long-term capital gains throughout the year and to distribute those gains on a regular monthly basis to shareholders. The Board of Directors of the Fund may amend, terminate or suspend the managed distribution policy at any time, which could have an adverse effect on the market price of the Fund's shares.
The Fund's m

In [21]:
sum_urls = filter_urls[0:10]
sum_urls

178     https://www.prnewswire.com:443/news-releases/c...
483     https://www.prnewswire.com:443/news-releases/c...
1059    https://www.prnewswire.com:443/news-releases/c...
1328    https://www.prnewswire.com:443/news-releases/c...
1506    https://www.prnewswire.com:443/news-releases/d...
1775    https://www.prnewswire.com:443/news-releases/n...
2527    https://www.prnewswire.com/news-releases/vodaf...
3123    https://www.prnewswire.com:443/news-releases/a...
3441    https://www.prnewswire.com:443/news-releases/c...
3570    https://www.prnewswire.com:443/news-releases/e...
Name: 4, dtype: object

In [23]:
from bs4 import BeautifulSoup
import requests
ls = []
for i in range(len(sum_urls)): #filter_urls
    URL = sum_urls.iloc[i] # filter_urls
    page = requests.get(URL,headers={'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    soup = BeautifulSoup(page.content, "html.parser")
    master = soup.find(id="main")
    final = master.find("div", class_="col-sm-10 col-sm-offset-1")
    print(URL)
    print(final.text)

https://www.prnewswire.com:443/news-releases/cohen--steers-quality-income-realty-fund-inc-rqi-notification-of-sources-of-distribution-under-section-19a-301434536.html

NEW YORK, Nov. 30, 2021 /PRNewswire/ -- This press release provides shareholders of Cohen & Steers Quality Income Realty Fund, Inc. (NYSE: RQI) (the "Fund") with information regarding the sources of the distribution to be paid on November 30, 2021 and cumulative distributions paid fiscal year-to-date. 
In December 2012, the Fund implemented a managed distribution policy in accordance with exemptive relief issued by the Securities and Exchange Commission. The managed distribution policy seeks to deliver the Fund's long-term total return potential through regular monthly distributions declared at a fixed rate per common share. The policy gives the Fund greater flexibility to realize long-term capital gains throughout the year and to distribute those gains on a regular monthly basis to shareholders. The Board of Directors o

https://www.prnewswire.com:443/news-releases/cohen--steers-select-preferred-and-income-fund-inc-psf-notification-of-sources-of-distribution-under-section-19a-301434534.html

NEW YORK, Nov. 30, 2021 /PRNewswire/ -- This press release provides shareholders of Cohen & Steers Select Preferred and Income Fund, Inc. (NYSE: PSF) (the "Fund") with information regarding the sources of the distribution to be paid on November 30, 2021 and cumulative distributions paid fiscal year-to-date.
In December 2016, the Fund implemented a managed distribution policy in accordance with exemptive relief issued by the Securities and Exchange Commission. The managed distribution policy seeks to deliver the Fund's long-term total return potential through regular monthly distributions declared at a fixed rate per common share. The policy gives the Fund greater flexibility to realize long-term capital gains throughout the year and to distribute those gains on a regular monthly basis to shareholders. The Board of 

https://www.prnewswire.com:443/news-releases/cohen--steers-total-return-realty-fund-inc-rfi-notification-of-sources-of-distribution-under-section-19a-301434540.html

NEW YORK, Nov. 30, 2021 /PRNewswire/ -- This press release provides shareholders of Cohen & Steers Total Return Realty Fund, Inc. (NYSE: RFI) (the "Fund") with information regarding the sources of the distribution to be paid on November 30, 2021 and cumulative distributions paid fiscal year-to-date.
In December 2011, the Fund implemented a managed distribution policy in accordance with exemptive relief issued by the Securities and Exchange Commission. The managed distribution policy seeks to deliver the Fund's long-term total return potential through regular monthly distributions declared at a fixed rate per common share. The policy gives the Fund greater flexibility to realize long-term capital gains throughout the year and to distribute those gains on a regular monthly basis to shareholders. The Board of Directors of the

https://www.prnewswire.com:443/news-releases/datadog-achieves-aws-graviton-ready-designation-301434535.html

NEW YORK, Nov. 30, 2021 /PRNewswire/ -- Datadog, Inc. (NASDAQ: DDOG), the monitoring and security platform for cloud applications, announced today that it has achieved the AWS Graviton Ready designation, part of the Amazon Web Services (AWS) Service Ready Program. This designation recognizes that Datadog has demonstrated successful integration with AWS Graviton.
Achieving the AWS Graviton Ready designation differentiates Datadog as an AWS Partner Network (APN) member with a product that robustly integrates with AWS Graviton and is generally available and fully supported for AWS customers. AWS Service Ready Partners have demonstrated success building products integrated with AWS services, helping AWS customers evaluate and use their technology productively, at scale, and with varying levels of complexity.
"Datadog is proud to achieve AWS Graviton Ready status," said Ilan Rabinovi

https://www.prnewswire.com:443/news-releases/amcor-unveils-state-of-the-art-healthcare-packaging-facility-in-singapore-to-serve-accelerating-customer-demands-in-asia-pacific-301434338.html

SINGAPORE and CHICAGO, Nov. 30, 2021 /PRNewswire/ -- Amcor (NYSE: AMCR) (ASX:AMC), a global leader in developing and producing responsible packaging, today announced the opening of its greenfield packaging facility in Tuas, Singapore. The dedicated healthcare packaging facility will address a current and future market need, as demand for healthcare packaging is rapidly increasing across the region.
Amcor's new Tuas facility comprises cleanroom manufacturing environments, capability in three-, seven- and nine-layer co-extrusion blown film, and state-of-the-art flexographic printing. It is also fully certified to ISO 13485, ensuring the ability to fully satisfy industry regulatory requirements.
"With the Asia Pacific healthcare packaging market expected to grow at a more than 8% through 2026*, Amcor i

https://www.prnewswire.com:443/news-releases/everest-medicines-and-providence-therapeutics-jointly-announce-vaccine-development-strategy-to-address-omicron-b1-1-529-sars-cov-2-variant-301434104.html

SHANGHAI and CALGARY, Canada, Nov. 30, 2021 /PRNewswire/ -- Everest Medicines and Providence Therapeutics ("Providence") jointly announced today that the companies have started working on a new version of COVID-19 vaccine specifically targeting the new Omicron variant. 
Scientists from the two companies have analyzed the sequence of the SARS-CoV-2 Omicron variant, selected viral sequences, and designed plasmid clones. We expect the new Omicron SARS-CoV-2 variant vaccines can be advanced into clinical testing in less than 100 days. 
"We believe it is essential to develop new vaccines quickly and effectively to combat the new Omicron COVID-19 variant. We will work closely with regulatory bodies and seek the appropriate clinical studies and regulatory approvals to bring this vaccine to market